<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/age_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies


In [0]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 19.6MB/s 


## Data Preparation

### Download Data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

train_data_json_id = '1CDMRQdAhcws_g1yDw_29ZD5DNDDyi7Xw'
test_data_json_id = '1_0dpT5HRTWocnK35KLQFDHzJiwV2-IQZ'

utk_images_7z_id = '1c61PoUhIPKeoRzB0XDI23XMDyJaCfKSh'
utk_landmarks_7z_id = '1Nxg7KKfEkDBWCqhusE1S6Edp6n3tTOuN'

appareal_labels_json_id = '1_zfGunGuqyrftDJIEKw6NVJOS55vyOrh'
appareal_images_7z_id = '1BDm6r88XLwDFsqOa2ZbbUtW1HDyHo5yA'
appareal_landmarks_7z_id = '1Am36Tk-BnjfV1d8_iUpRcW-cPfQtAN0H'

wiki_labels_json_id = '1BamAqN3tNEMh6kNQQ4C8nWf6gOA2IS6X'
wiki_images_7z_id = '1Fy3pi-Pra1IsN9HDD268nRvXa1TbsryE'
wiki_landmarks_7z_id = '1M-YeSGEEboVqNK8pTCJhbxeVaLp0TKJ4'

if not os.path.exists('./data'):
  os.makedirs('./data')
if not os.path.exists('./data/utk'):
  os.makedirs('./data/utk')
if not os.path.exists('./data/appareal'):
  os.makedirs('./data/appareal')
if not os.path.exists('./data/wiki'):
  os.makedirs('./data/wiki')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
    
print('downloading trainData.json and testData.json ...')
drive.CreateFile({ 'id': train_data_json_id }).GetContentFile('./data/trainData.json')
drive.CreateFile({ 'id': test_data_json_id }).GetContentFile('./data/testData.json')

print('downloading utk data ...')
drive.CreateFile({ 'id': utk_images_7z_id }).GetContentFile('./data/utk/images.7z')
drive.CreateFile({ 'id': utk_landmarks_7z_id }).GetContentFile('./data/utk/landmarks.7z')

print('downloading appareal data ...')
drive.CreateFile({ 'id': appareal_labels_json_id }).GetContentFile('./data/appareal/labels.json')
drive.CreateFile({ 'id': appareal_images_7z_id }).GetContentFile('./data/appareal/images.7z')
drive.CreateFile({ 'id': appareal_landmarks_7z_id }).GetContentFile('./data/appareal/landmarks.7z')

print('downloading wiki data ...')
drive.CreateFile({ 'id': wiki_labels_json_id }).GetContentFile('./data/wiki/labels.json')
drive.CreateFile({ 'id': wiki_images_7z_id }).GetContentFile('./data/wiki/images.7z')
drive.CreateFile({ 'id': wiki_landmarks_7z_id }).GetContentFile('./data/wiki/landmarks.7z')
  
print('done!')

KeyboardInterrupt: ignored

### Unzip Data

In [0]:
!cd ./data/utk && p7zip -d ./images.7z
!cd ./data/utk && p7zip -d ./landmarks.7z
!cd ./data/appareal && p7zip -d ./images.7z
!cd ./data/appareal && p7zip -d ./landmarks.7z
!cd ./data/wiki && p7zip -d ./images.7z
!cd ./data/wiki && p7zip -d ./landmarks.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 216371463 bytes (207 MiB)

Extracting archive: ./images.7z
--
Path = ./images.7z
Type = 7z
Physical Size = 216371463
Headers Size = 309426
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% 1        2% 380 - cropped-images/12_1_4_20170103200804119.jpg.chip_0.jpg                                                                   3% 437 - cropped-images/13_1_0_20170109204849625.jpg.chip_0.jpg                                                                 

## Training

### Imports

In [0]:
import cv2
import math
import json
import random
import time
import os
import numpy as np
import tensorflow as tf

### Preprocessing

In [0]:
def num_in_range(val, min_val, max_val):
  return min(max(min_val, val), max_val)

def random_crop(img, landmarks):
  height, width, _ = img.shape
  min_x, min_y, max_x, max_y = width, height, 0, 0
  for pt in landmarks:
    min_x = pt['x'] if pt['x'] < min_x else min_x
    min_y = pt['y'] if pt['y'] < min_y else min_y
    max_x = max_x if pt['x'] < max_x else pt['x']
    max_y = max_y if pt['y'] < max_y else pt['y']
  
  min_x = int(num_in_range(min_x, 0, 1) * width)
  min_y = int(num_in_range(min_y, 0, 1) * height)
  max_x = int(num_in_range(max_x, 0, 1) * width)
  max_y = int(num_in_range(max_y, 0, 1) * height)
  x0 = random.randint(0, min_x)
  y0 = random.randint(0, min_y)
  x1 = random.randint(0, abs(width - max_x)) + max_x
  y1 = random.randint(0, abs(height - max_y)) + max_y

  return img[y0:y1, x0:x1]

def resize_preserve_aspect_ratio(img, size):
  height, width, _ = img.shape
  max_dim = max(height, width)
  ratio = size / float(max_dim)
  shape = (height * ratio, width * ratio)
  resized_img = cv2.resize(img, (int(round(height * ratio)), int(round(width * ratio))))
  
  return resized_img
  
def pad_to_square(img):
  height, width, channels = img.shape
  max_dim = max(height, width)
  square_img = np.zeros([max_dim, max_dim, channels])

  dx = math.floor(abs(max_dim - width) / 2)
  dy = math.floor(abs(max_dim - height) / 2)
  square_img[dy:dy + height,dx:dx + width] = img

  return square_img

def preprocess(img, landmarks, size):
  cropped_img = random_crop(img, landmarks)
  resized_img = resize_preserve_aspect_ratio(cropped_img, size)
  square_img = pad_to_square(resized_img)
  
  return square_img

### Neural Network

In [0]:
def conv2d(x, weights, stride):
  out = tf.nn.conv2d(x, weights['filter'], stride, 'SAME')
  out = tf.add(out, weights['bias'])
  return out

def depthwise_separable_conv2d(x, weights, stride):
  out = tf.nn.separable_conv2d(x, weights['depthwise_filter'], weights['pointwise_filter'], stride, 'SAME')
  out = tf.add(out, weights['bias'])
  return out
  
def fully_connected(x, weights):
  out = tf.reshape(x, [-1, weights['weights'].get_shape().as_list()[0]])
  out = tf.matmul(out, weights['weights'])
  out = tf.add(out, weights['bias'])
  return out

def dense_block(x, weights, is_first_layer = False, is_scale_down = True):
  initial_stride = [1, 2, 2, 1]  if is_scale_down else [1, 1, 1, 1]
  out1 = conv2d(x, weights['conv0'], initial_stride) if is_first_layer else depthwise_separable_conv2d(x, weights['conv0'], initial_stride)
  
  in2 = tf.nn.relu(out1)
  out2 = depthwise_separable_conv2d(in2, weights['conv1'], [1, 1, 1, 1])

  in3 = tf.nn.relu(tf.add(out1, out2))
  out3 = depthwise_separable_conv2d(in3, weights['conv2'], [1, 1, 1, 1])

  in4 = tf.nn.relu(tf.add(out1, tf.add(out2, out3)))
  out4 = depthwise_separable_conv2d(in4, weights['conv3'], [1, 1, 1, 1])

  return tf.nn.relu(tf.add(out1, tf.add(out2, tf.add(out3, out4))))

def normalize(x, mean_rgb):
  r, g, b = mean_rgb
  shape = np.append(np.array(x.shape[0:3]), [1])
  avg_r = tf.fill(shape, r)
  avg_g = tf.fill(shape, g)
  avg_b = tf.fill(shape, b)
  avg_rgb = tf.concat([avg_r, avg_g, avg_b], 3)

  return tf.divide(tf.subtract(x, avg_rgb), 255)

def forward(batch_tensor, weights):
  mean_rgb = [122.782, 117.001, 104.298]
  normalized = normalize(batch_tensor, mean_rgb)

  out = dense_block(normalized, weights['dense0'], True)
  out = dense_block(out, weights['dense1'])
  out = dense_block(out, weights['dense2'])
  out = dense_block(out, weights['dense3'])
  out = tf.nn.avg_pool(out, [1, 7, 7, 1], [1, 2, 2, 1], 'VALID')
  out = fully_connected(out, weights['fc_age'])

  return out



### Weight Serialization

In [0]:
class WeightProcessor:
  def __init__(self, process_weights, processor_bias):
    self.process_weights = process_weights
    self.processor_bias = processor_bias
  
  def process_conv_weights(self, channels_in, channels_out, prefix):
    self.process_weights([3, 3, channels_in, channels_out], prefix + '/filter')
    self.processor_bias([channels_out], prefix + '/bias')

  def process_depthwise_separable_conv2d_weights(self, channels_in, channels_out, prefix):
    self.process_weights([3, 3, channels_in, 1], prefix + '/depthwise_filter'),
    self.process_weights([1, 1, channels_in, channels_out], prefix + '/pointwise_filter'),
    self.processor_bias([channels_out], prefix + '/bias')

  def process_dense_block_weights(self, channels_in, channels_out, prefix, is_first_layer = False):
    conv0_processor = self.process_conv_weights if is_first_layer else self.process_depthwise_separable_conv2d_weights
    conv0_processor(channels_in, channels_out, prefix + '/conv0')
    self.process_depthwise_separable_conv2d_weights(channels_out, channels_out, prefix + '/conv1')
    self.process_depthwise_separable_conv2d_weights(channels_out, channels_out, prefix + '/conv2')
    self.process_depthwise_separable_conv2d_weights(channels_out, channels_out, prefix + '/conv3')

  def process(self):
    self.process_dense_block_weights(3, 32, 'dense0', True)
    self.process_dense_block_weights(32, 64, 'dense1')
    self.process_dense_block_weights(64, 128, 'dense2')
    self.process_dense_block_weights(128, 256, 'dense3')
    self.process_weights([256, 1], 'fc_age/weights')
    self.processor_bias([1], 'fc_age/bias')

class WeightMap:
  def __init__(self, tensors, tensor_paths):
    self.weights = {}
    for idx, tensor in enumerate(tensors):
      tensor_path = tensor_paths[idx]

      tmp = self.weights
      keys = tensor_path.split('/')
      for path_idx, key in enumerate(keys):
        is_end = path_idx == len(keys) - 1
        tmp[key] = tensor if is_end else (tmp[key] if key in tmp else {})
        tmp = tmp[key]
  
  def get_tensor_from_path(self, tensor_path):
    tmp = self.weights
    for key in tensor_path.split('/'):
      tmp = tmp[key]
    return tmp
  
  def set_tensor_from_path(self, tensor_path, tensor):
    tmp = self.weights
    path = tensor_path.split('/')
    for key in path[0: len(path) - 1]:
      tmp = tmp[key]
    tmp[path[len(path) - 1]] = tensor
  
  def trainable(self):
    def to_trainable(shape, tensor_path):
      self.set_tensor_from_path(tensor_path, tf.Variable(self.get_tensor_from_path(tensor_path)))
    
    WeightProcessor(to_trainable, to_trainable).process()
    
    
def init_weights(weight_initializer = tf.keras.initializers.glorot_normal(), bias_initializer = tf.zeros):
  tensors = []
  tensor_paths = []
  def process_weights(shape, tensor_path):
    tensors.append(weight_initializer(shape))
    tensor_paths.append(tensor_path)
  def process_bias(shape, tensor_path):
    tensors.append(bias_initializer(shape))
    tensor_paths.append(tensor_path)

  WeightProcessor(process_weights, process_bias).process()

  return WeightMap(tensors, tensor_paths)
  
def load_weights(checkpoint_file):  
  checkpoint_data = np.load(checkpoint_file)
  
  idx = 0
  tensors = []
  tensor_paths = []
  def extract_weights_from_shape(shape, tensor_path):
    nonlocal idx
    size = 1
    for val in shape:
      size = size * val
    tensor = tf.convert_to_tensor(np.reshape(checkpoint_data[idx:idx + size], shape), dtype=tf.float32)
    
    idx += size
    tensors.append(tensor)
    tensor_paths.append(tensor_path)

  WeightProcessor(extract_weights_from_shape, extract_weights_from_shape).process()

  return WeightMap(tensors, tensor_paths)

def save_weights(weight_map, checkpoint_file):  
  checkpoint_data = np.array([])
  def append_weights(shape, tensor_path):
    nonlocal checkpoint_data
    tensor_data_flat = weight_map.get_tensor_from_path(tensor_path).eval().flatten()
    checkpoint_data = np.append(checkpoint_data, tensor_data_flat)

  WeightProcessor(append_weights, append_weights).process()
  np.save(checkpoint_file, checkpoint_data)

### Data Loader

In [0]:
def load_json(json_file_path):
  with open(json_file_path) as json_file:  
    return json.load(json_file)

def load_image_batch(datas):
  preprocessed_imgs = []
  
  for data in datas:
    db = data['db']
    img_file = data['file']
    file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
    landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
    img_file_path = './data/' + db + '/cropped-images/' + img_file
    landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file
    
    img = cv2.imread(img_file_path)
    if img is None:
      raise 'failed to read image from path: ' + img_file_path
      
    landmarks = load_json(landmarks_file_path)
    preprocessed_img = preprocess(img, landmarks, 112)
    preprocessed_imgs.append(preprocessed_img)
      
  return np.stack(preprocessed_imgs, axis=0)

def shuffle_array(arr):
  arr_clone = arr[:]
  random.shuffle(arr_clone)
  return arr_clone
  
class DataLoader:
  def __init__(self, start_epoch):
    self.train_data = load_json('./data/trainData.json')
    self.buffered_data = shuffle_array(self.train_data)
    self.appareal_labels = load_json('./data/appareal/labels.json')
    self.wiki_labels = load_json('./data/wiki/labels.json')
    self.current_idx = 0
    self.epoch = start_epoch
 
  def get_end_idx(self):
    return len(self.buffered_data)
    
  def lookup_labels(self, datas):
    labels = []

    for data in datas:
      db = data['db']
      img_file = data['file']
      
      if db == 'utk':
        age = int(float(img_file.split('_')[0]))
        labels.append(age)
      elif db == 'appareal':
        age = self.appareal_labels[img_file]['age']
        labels.append(age)
      elif db == 'wiki':
        age = self.wiki_labels[img_file]['age']
        labels.append(age)
      else: raise('unknown db: ' + db)
        
    return np.expand_dims(np.stack(labels, axis = 0), axis = 1)
    
  def next_batch(self, batch_size):  
    from_idx = self.current_idx
    to_idx = self.current_idx + batch_size
    
    # end of epoch
    if (to_idx > len(self.buffered_data)):
      self.epoch += 1
      self.buffered_data = self.buffered_data[from_idx:] + shuffle_array(self.train_data)  
      from_idx = 0
      to_idx = batch_size
      
    self.current_idx = to_idx
    
    next_data = self.buffered_data[from_idx:to_idx]
      
    batch_x = load_image_batch(next_data)
    batch_y = self.lookup_labels(next_data)
    
    return batch_x, batch_y

### Training

In [0]:
#tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# training parameters
learning_rate = 0.001
start_epoch = 22
end_epoch = 2000
batch_size = 32

X = tf.placeholder(tf.float32, [batch_size, 112, 112, 3])
Y = tf.placeholder(tf.float32, [batch_size, 1])

weight_map = init_weights() if start_epoch == 0 else load_weights('./checkpoint_epoch_' + str(start_epoch - 1) + '.npy')
weight_map.trainable()

data_loader = DataLoader(start_epoch)

age = forward(X, weight_map.weights)

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
loss_op = tf.reduce_mean(tf.abs(tf.subtract(age, Y)))
train_op = optimizer.minimize(loss_op)

init = tf.global_variables_initializer()

log_file = open('./log.txt', 'w') 

total_loss = 0
iteration_count = 0
ts_epoch = time.time()
#with tf.Session(tpu_address) as sess:
with tf.device('/gpu:0'):
  with tf.Session(config=config) as sess:
    sess.run(init)

    while data_loader.epoch <= end_epoch:
      epoch = data_loader.epoch
      current_idx = data_loader.current_idx
      end_idx = data_loader.get_end_idx()

      ts = time.time()

      batch_x, batch_y = data_loader.next_batch(batch_size)
      loss, _ = sess.run([loss_op, train_op], feed_dict = { X: batch_x, Y: batch_y })
      total_loss += loss
      iteration_count += 1
      log_file.write("epoch " + str(epoch) + ", (" + str(current_idx) + " of " + str(end_idx) + "), loss= " + "{:.4f}".format(loss) 
            + ", time= " + str((time.time() - ts) * 1000) + "ms \n")
      if epoch != data_loader.epoch:
        print('next epoch: ' + str(data_loader.epoch))

        save_weights(weight_map, './checkpoint_epoch_' + str(epoch) + '.npy')

        epoch_txt = open('epoch_' + str(epoch) + '.txt', 'w')
        epoch_txt.write('total_loss= ' + str(total_loss) + '\n')
        epoch_txt.write('avg_loss= ' + str(total_loss / iteration_count) + '\n')
        epoch_txt.write('learning_rate= ' + str(learning_rate) + '\n')
        epoch_txt.write('batch_size= ' + str(batch_size) + '\n')
        epoch_txt.write('epoch_time= ' + str(time.time() - ts_epoch) + 's \n')
        epoch_txt.close()

        total_loss = 0
        iteration_count = 0              
        ts_epoch = time.time()

    print('done!')
    log_file.close() 